In [1]:
import pandas as pd
import numpy as np
import os
import glob

# 처리할 CSV 파일들이 위치한 폴더 지정
input_folder = './'  # 현재 디렉토리 기준 (필요시 수정)
csv_files = glob.glob(os.path.join(input_folder, '*.csv'))

for file_path in csv_files:
    # 원래 파일 이름 추출
    base_name = os.path.basename(file_path)
    name_without_ext = os.path.splitext(base_name)[0]

    # CSV 로드
    df = pd.read_csv(file_path)

    # timestamp offset 계산
    df['timestamp_offset'] = df['timestamp'] - df['timestamp'].min()

    # 100ms 단위 bin 생성
    df['time_bin'] = (df['timestamp_offset'] // 100).astype(int)

    # 수치형 컬럼과 범주형 컬럼 구분
    numeric_cols = df.select_dtypes(include=[np.number]).columns.difference(['time_bin'])
    categorical_col = 'direction'

    # 집계 함수 정의
    agg_funcs = {col: 'mean' for col in numeric_cols}
    agg_funcs[categorical_col] = lambda x: x.mode().iloc[0] if not x.mode().empty else None

    # 그룹화 및 집계
    aggregated_df = df.groupby('time_bin').agg(agg_funcs).reset_index(drop=True)

    # 출력 파일 경로 생성
    output_path = os.path.join(input_folder, f"avg_100ms_{name_without_ext}.csv")

    # 저장
    aggregated_df.to_csv(output_path, index=False)
    print(f"Saved: {output_path}")


Saved: ./avg_100ms_kevin_S22ultra_pedLog_1754296294156.csv
